## 使用networkx计算PageRank

In [17]:
import networkx as nx

# 创建一个有向图
G = nx.DiGraph()

### 读取mysql数据库

In [18]:
import pymysql
cnx = pymysql.connect(host='localhost', user='root', password='123456')
cursor = cnx.cursor()
cnx.select_db('IR_db')

In [19]:
def get_douban_url_as_node():
    # 创建SQL查询
    sql = "SELECT url FROM douban"
    # 执行查询
    cursor.execute(sql)
    # 获取所有的结果
    results = cursor.fetchall()
    # 将结果从元组列表转换为普通列表，并添加到图中作为节点
    for result in results:
        G.add_node(result[0])


In [20]:

def get_ids_from_same_title():
    # 创建SQL查询
    sql = "SELECT page_id FROM same_title"
    # 执行查询
    cursor.execute(sql)
    # 获取所有的结果
    results = cursor.fetchall()
    # 将结果从元组列表转换为普通列表
    ids_same_title = [result[0] for result in results]
    return ids_same_title

def get_page_url_as_node():
    global ids_page
    # 创建SQL查询
    sql = "SELECT id,url FROM page"
    # 执行查询
    cursor.execute(sql)
    # 获取所有的结果
    results = cursor.fetchall()
    # 将结果从元组列表转换为普通列表
    ids = [result[0] for result in results]
    urls = [result[1] for result in results]
    # 获取相同title的id
    ids_same_title = get_ids_from_same_title()
    print(len(ids_same_title))
    # 去除相同title的id
    for id,url in zip(ids,urls):
        if id not in ids_same_title:
            G.add_node(url)

    # 从same_title中按title分组，获取每组的第一个id
    sql = "SELECT MIN(page_id) FROM same_title GROUP BY title"
    cursor.execute(sql)
    results = cursor.fetchall()
    ids = [result[0] for result in results]
    print(len(ids))
    # 找到这些id对应的url
    for id in ids:
        sql = "SELECT url FROM page WHERE id = %d" % id
        cursor.execute(sql)
        results = cursor.fetchall()
        url = results[0]
        G.add_node(url)

### 创建节点 × 没必要了，创建边的时候会自动创建节点

In [21]:
get_douban_url_as_node()

In [22]:
get_page_url_as_node()

21606
2283


### 创建节点之间的边

In [23]:
def get_douban_edges():
    # 从数据库中获取所有的记录
    sql = "SELECT url, links FROM douban"
    cursor.execute(sql)
    results = cursor.fetchall()
     # 对于每一条记录
    for url, links in results:
        # 将links按照'\n'解析成一个列表
        links = links.split('\n')
        # 对于每一个link
        for link in links:
           G.add_edge(url, link)

In [24]:
get_douban_edges()

In [25]:
def create_url_to_url_mapping():
    # 创建一个空的映射
    url_to_url = {}
    # 从same_title表中按title分组，获取每组的第一个URL
    sql = "SELECT MIN(page_id), title FROM same_title GROUP BY title"
    cursor.execute(sql)
    results = cursor.fetchall()
    # 将每个title的所有URL映射到对应的节点
    for page_id, title in results:
        # 获取该title的第一个URL
        sql = "SELECT url FROM same_title WHERE page_id = %d" % page_id
        cursor.execute(sql)
        results = cursor.fetchall()
        target_url = results[0]
        # 获取该title的所有URL
        sql = "SELECT url FROM same_title WHERE title = %s"
        cursor.execute(sql,(title,))
        urls = cursor.fetchall()
        for url in urls:
            url_to_url[url] = target_url
    return url_to_url

In [26]:
# 拿到同title的url映射
url_to_url = create_url_to_url_mapping()

In [27]:
def get_page_edges():
    # 从数据库中获取所有的记录
    sql = "SELECT url, links FROM page"
    cursor.execute(sql)
    results = cursor.fetchall()
    # 对于每一条记录
    for url, links in results:
        # 如果URL在映射中，使用映射中的值替换URL
        url = url_to_url.get(url, url)
        # 将links按照'\n'解析成一个列表
        links = links.split('\n')
        # 对于每一个link
        for link in links:
            # 如果link在映射中，使用映射中的值替换link
            link = url_to_url.get(link, link)
            # 在图G中添加一条从url到link的边
            G.add_edge(url, link)

In [28]:
get_page_edges()

### 计算PageRank

In [29]:
def cal_page_rank():
    # 使用networkx中的pagerank函数计算PR值
    pr = nx.pagerank(G)
    return pr

### 更新es索引

In [30]:
from elasticsearch import Elasticsearch

# 实例化es
es = Elasticsearch(hosts="http://localhost:9200")

In [31]:
from elasticsearch.helpers import scan

def add_page_rank_to_documents(pr):
    # 遍历web索引中的所有文档
    for doc in scan(es, index='web'):
        # 获取文档的URL
        url = doc['_source']['url']
        # 在pr字典中查询对应的PageRank值
        page_rank = pr.get(url)
        # 如果找到了PageRank值
        if page_rank is not None:
            # 更新文档，添加pageRank字段
            es.update(index='web', id=doc['_id'], body={'doc': {'pageRank': page_rank}})
        else :
            # 否则，设置pageRank字段为0
            es.update(index='web', id=doc['_id'], body={'doc': {'pageRank': 0}})

pr = cal_page_rank()
add_page_rank_to_documents(pr)
## 打印PageRank值
# 获取web索引中的前一百条文档
res = es.search(index="web", body={"query": {"match_all": {}}}, size=100)

# 遍历这些文档
for doc in res['hits']['hits']:
    # 获取并打印文档的title和pageRank字段
    title = doc['_source'].get('title', 'Not available')
    page_rank = doc['_source'].get('pageRank', 'Not available')
    print(f"Title: {title}, PageRank: {page_rank}")

C:\Users\89214\AppData\Local\Temp\ipykernel_19580\3337885599.py:5: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  for doc in scan(es, index='web'):
C:\Users\89214\AppData\Local\Temp\ipykernel_19580\3337885599.py:13: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  es.update(index='web', id=doc['_id'], body={'doc': {'pageRank': page_rank}})
C:\Users\89214\AppData\Local\Temp\ipykernel_19580\3337885599.py:16: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://

Title: 通俗、真诚、务实。所谓转型，一言蔽之，到了真正“以人（民）为本”的时候了。, PageRank: 2.787516544936364e-05
Title: 《置身事内》思维导图整理, PageRank: 2.787516544936364e-05
Title: 记录3月11日五道口PAGEONE书店兰小欢老师签售讲座部分内容, PageRank: 2.787516544936364e-05
Title: 《置身事内》每章末的推荐阅读书单汇总, PageRank: 2.787516544936364e-05
Title: 兰小欢谈地方政府与经济发展, PageRank: 2.787516544936364e-05
Title: 一本书读懂当代中国政治与经济，4.5星, PageRank: 2.787516544936364e-05
Title: 置身事内, PageRank: 0.00016955194515546433
Title: 江湖儿女，何处恋爱？师兄弟妹，近水楼台, PageRank: 1.6093518870302003e-05
Title: 独一无二的精致男人——黄药师, PageRank: 1.6093518870302003e-05
Title: 射雕英雄传：依稀往梦似曾见, PageRank: 1.6093518870302003e-05
Title: 郭靖武功随笔, PageRank: 1.6093518870302003e-05
Title: 蓉儿正当年少, PageRank: 1.6093518870302003e-05
Title: “我们骑墙派难做啊！！！”——论金庸男主永恒的难题, PageRank: 1.6093518870302003e-05
Title: 金庸杂谈--射雕英雄传, PageRank: 1.6093518870302003e-05
Title: 射雕英雄传, PageRank: 5.042628906783961e-05
Title: 对友谊最大的误解，就是认为它是万能的, PageRank: 2.787516544936364e-05
Title: 爱的炮灰——《追风筝的人》读后感, PageRank: 2.787516544936364e-05
Title: 阿富汗版《兄弟》, PageRank: 

C:\Users\89214\AppData\Local\Temp\ipykernel_19580\3337885599.py:22: DeprecationWarning: Received 'size' via a specific parameter in the presence of a 'body' parameter, which is deprecated and will be removed in a future version. Instead, use only 'body' or only specific parameters.
  res = es.search(index="web", body={"query": {"match_all": {}}}, size=100)
C:\Users\89214\AppData\Local\Temp\ipykernel_19580\3337885599.py:22: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  res = es.search(index="web", body={"query": {"match_all": {}}}, size=100)


In [36]:
"""
res = es.search(index="web", body={"query": {"match_all": {}}})
for doc in res['hits']['hits']:
    print(doc['_source'])  # 检查每个文档是否包含 pageRank 字段
"""

# 使用URL搜索文档
# res = es.search(index="web", body={"query": {"match": {"url": 'https://www.runoob.com/'}}})
res = es.search(index="web", body={"query": {"match": {"url": 'https://edu.cnblogs.com/'}}})

# 查看page_id
page_id = res['hits']['hits'][0]['_source']['pageRank']
print(page_id)

C:\Users\89214\AppData\Local\Temp\ipykernel_19580\1599606075.py:9: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  res = es.search(index="web", body={"query": {"match": {"url": 'https://edu.cnblogs.com/'}}})


IndexError: list index out of range

### 测试pageRank影响排序结果

In [37]:
def deal_bool_query(query):
    # 执行搜索
    response = es.search(
        index="web",
        body={
            "query": {
                "function_score": {
                    "query": {
                        "bool": {
                            "must": [
                                {
                                    "multi_match": {
                                        "query": query,
                                        "fields": ["title", "content"]
                                    }
                                }
                            ]
                        }
                    },
                    "script_score": {
                        "script": {
                            "source": "doc['pageRank'].value == 0 ? 1 : Math.log1p(doc['pageRank'].value * params.factor)",
                            "params": {
                                "factor": 100000
                            }
                        }
                    },
                    "boost_mode": "sum"
                }
            },
            "explain": True  # 添加这一行
        }
    )
    # 获取搜索结果
    hits = response["hits"]["hits"]
    # 格式化搜索结果
    results = [
        {
            "id": hit["_id"],
            "title": hit["_source"]["title"],
            "content": hit["_source"]["content"],
            "url": hit["_source"]["url"],
            "type": hit["_source"]["type"],
            "pageRank": hit["_source"]["pageRank"],
            "explanation": hit["_explanation"]  # 添加这一行
        }
        for hit in hits
    ]
    return results

In [38]:
results = deal_bool_query('中国恒大于2022年3月21日起停牌')

for result in results:
    print(result['title'], result['url'], result['type'])
    print(result['explanation']['value'])
    print(result['pageRank'])
    

此次，中国恒大停牌距其复牌仅一个月。此前，中国恒大于2022年3月21日起停牌，在经历了停牌超17个月后，中国恒大宣布于今年8月28日起重新复牌。 https://www.thepaper.cn/newsDetail_forward_24776842 news
56.73377
0
两年净亏损8120亿-总负债超24万亿中国恒大已资不抵债 https://www.jiemian.com/article/9760788.html news
46.87542
0
国资房企远洋集团爆雷半年巨亏近200亿票据-债券也濒临违约 http://www.stcn.com/article/detail/947042.html news
37.592705
0
中国恒大将于周一在香港复牌 https://www.guancha.cn/GongSi/2023_08_26_706438.shtml news
35.624535
0
中国恒大-恒大汽车和恒大物业在港交所集体停牌 http://03333.HK news
31.351944
0
中东战投暂停资金支持-恒大汽车复牌大跌 http://NWTN.US news
29.703127
0
恒大复牌-涨幅明显 https://k.sina.com.cn/article_5044281310_12ca99fde019021kub.html news
27.169312
0
格力地产与珠免重启资产重组-此前因涉内幕交易案暂停两年半 https://www.caixin.com/2022-12-03/101974009.html news
25.65512
0
中国恒大复牌首日-盘中跌90 https://wallstreetcn.com/articles/3696497 news
25.254686
0
许家印的香港恒大中心更名为万通保险中心-系马云旗下公司 https://www.nbd.com.cn/articles/2023-06-26/2886217.html news
23.644526
0


C:\Users\89214\AppData\Local\Temp\ipykernel_19580\3839144991.py:3: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  response = es.search(


In [39]:
import math

print(math.log1p(0.00039118477103319317*100000))
print(math.log1p(3.9509503613941765e-05*100000))

print(math.log1p(0.00039118477103319317*10000000))
print(math.log1p(3.9509503613941765e-05*10000000))

3.691837003831898
1.5995795503532988
8.272020702549892
5.9816541727124815
